Quickstart
==

This notebook demonstrates how to use `mutyper` for computing mutation type data for VCF/BCF data

## Python API demo

In [ ]:
import mutyper

Path to an ancestral FASTA file for human chromosome 1

In [ ]:
fasta = '../example_data/ancestor.fa'

Show the first 3 lines with a bash cell

In [ ]:
!head -3 {fasta}

The `mutyper` package has a single class `Ancestor`. We instantiate an `Ancestor` object using our FASTA file
- We're interested in 3-mer context, which we can indicate with the `k` keyword argument (by default it will be 3 though).

In [ ]:
ancestor = mutyper.Ancestor(fasta, k=3)

We can inspect FASTA record names, showing that the FASTA contains two records, chromosome `'1'` and `'2'`.

In [ ]:
ancestor.keys()

In [ ]:
ancestor['1']

and we can Pythonically slice sequences via fast random access without loading into memory (leaning on [pyfaidx](https://pythonhosted.org/pyfaidx/) under the hood):

In [ ]:
start = 100
end = 300
ancestor['1'][start:end]

We can also access these `FastaRecord` slices as string-like biopython `Seq` objects

In [ ]:
ancestor['1'][start:end].seq

The `mutation_type` method allows us to specify a SNP by the usual CHROM, POS, REF, ALT, and returns the correctly polarized mutation type as a tuple of ancestral kmer and derived kmer.

First, consider the triplet context at site 100 on chromosome 1:

In [ ]:
ancestor['1'][98:101]

Suppose a biallelic SNP at this site segregates in a population with reference allele T and alternative allele A. The mutation type of this SNP is

In [ ]:
ancestor.mutation_type('1', 99, 'T', 'A')

Note that, by default, mutation types are collapsed by reverse complementation such that the ancestral state at the target site is A or C. So in the above, the ancestral state T caused the ancestral triplet CTG to be reversed and complemented to CAG.

If the reference and alternative states are both distinct from the ancestral state, the site is not biallelic (an infinite sites violation), and the mutation type is returned as ambiguous.

In [ ]:
ancestor.mutation_type('1', 99, 'C', 'A')

The `region_contexts` method returns a generator of ancestral contexts (by default collapsed as described above) over the positions in a region specified as in BED file format CHROM, START, END

In [ ]:
start = 50
end = 100
for context in ancestor.region_contexts('1', start, end):
    print(context, end=' ')

Note that FASTA sites that are not nucleotide characters have non-identified ancestral state, so context is `None`. 

This generator method is used by the the `targets` method to compute the masked genomic target size for each $k$-mer context from a BED mask file. This may be useful for normalizing spectra in different genomic regions, or calibrating mutation rates.

In [ ]:
bed = '../example_data/mask.bed'

In [ ]:
!cat {bed}

Under the hood it loops over BED file entries and updates a `Counter` object for the different triplets

In [ ]:
ancestor.targets(bed)

We can also call it without the BED file mask to compute the unmasked target sizes of the ancestral FASTA

In [ ]:
ancestor.targets()

## Command line interface demo

Note that command line cells begin with `!` in Jupyter notebooks.

Display usage information for the mutyper command

In [ ]:
!mutyper -h

### `variants` subcommand

Usage:

In [ ]:
!mutyper variants -h

Path to a truncated VCF file for chromosome 1 from the 1000 Genomes Project.

In [ ]:
vcf = '../example_data/snps.vcf'

We'll use [bcftools](http://samtools.github.io/bcftools/bcftools.html#view) to display the first few variants, omitting header and genotype information.

In [ ]:
!bcftools view -HG {vcf} | head -5

Now we use the `variants` subcommand, and pipe to `bcftools` for display. Notice there is an additional INFO field `mutation_type`, and only biallelic SNPs are output. Also note that REF/ALT state and INFO/AC fields are polarized according to ancestral/derived allele.

In [ ]:
!mutyper variants {fasta} {vcf} | bcftools view -HG | head -5

### `targets` subcommand

Usage:

In [ ]:
!mutyper targets -h

In [ ]:
!mutyper targets {fasta} | head

The optional argument `--bed` resticts target size computation based on a BED mask file, and is useful of normalizing mutation spectra according to genomic region, or calibrating mutaiton rates.

In [ ]:
!mutyper targets {fasta} --bed {bed} | head

### `spectra` subcommand

Usage:

In [ ]:
!mutyper spectra -h

We first add mutation type information with the `variants` subcommand, then pipe into the `spectra` subcommand to generate mutation spectra for all samples, and finally crop the output with `head` and `cut` for display.

In [ ]:
!mutyper variants {fasta} {vcf} | mutyper spectra - | head | cut -f-5

Use the `--population` to get the spectrum for whole population, rather than each individual

In [ ]:
!mutyper variants {fasta} {vcf} | mutyper spectra - --population | head | cut -f-5

### `ksfs` subcommand

Usage:

In [ ]:
!mutyper ksfs -h

Similar to the previous subcommand, but now we are generating a sample frequency spectrum (SFS) for each mutation type.

In [ ]:
!mutyper variants {fasta} {vcf} | mutyper ksfs - | head | cut -f-5